# Signal Transformations

## Discrete Cosine Transformations

In [ ]:
from vitalDSP.transforms.discrete_cosine_transform import DiscreteCosineTransform
import numpy as np
import os
from vitalDSP.notebooks import process_in_chunks, plot_trace


FILE_PATH = os.path.join("..","..","..","sample_data","public","ecg.csv")
signal_col, date_col = process_in_chunks(FILE_PATH,data_type='ecg', fs=256)
signal_col = np.array(signal_col)

dct = DiscreteCosineTransform(signal_col)
dct_coefficients = dct.compute_dct()
compressed_coefficients = dct.compress_signal(threshold=0.05)
reconstructed_signal = dct.compute_idct(compressed_coefficients)
plot_trace(signal_col,reconstructed_signal)

# Feature extraction: Energy distribution in low-frequency components
low_freq_energy = np.sum(dct_coefficients[:20]**2)
total_energy = np.sum(dct_coefficients**2)
low_freq_ratio = low_freq_energy / total_energy

print("Low Frequency Energy Ratio:", low_freq_ratio)

In [ ]:
from vitalDSP.transforms.discrete_cosine_transform import DiscreteCosineTransform
import numpy as np
import os
from vitalDSP.notebooks import process_in_chunks, plot_trace


FILE_PATH = os.path.join("..","..","..","sample_data","public","ppg.csv")
signal_col, date_col = process_in_chunks(FILE_PATH,data_type='ppg', fs=100)
signal_col = np.array(signal_col)

dct = DiscreteCosineTransform(signal_col)
dct_coefficients = dct.compute_dct()

# Zero out small coefficients (noise reduction)
compressed_coefficients = dct.compress_signal(threshold=0.1)

# Reconstruct the signal from filtered DCT coefficients
ppg_reconstructed = dct.compute_idct(compressed_coefficients, norm='ortho')

plot_trace(signal_col, ppg_reconstructed)

## Fourier Transform

In [ ]:
from vitalDSP.transforms.fourier_transform import FourierTransform
import numpy as np
import matplotlib.pyplot as plt
import os
from vitalDSP.notebooks import process_in_chunks, plot_trace


FILE_PATH = os.path.join("..","..","..","sample_data","public","ecg.csv")
signal_col, date_col = process_in_chunks(FILE_PATH,data_type='ecg', fs=256)
signal_col = np.array(signal_col)

ft = FourierTransform(signal_col)
filtered_signal = ft.filter_frequencies(low_cutoff=0.3, high_cutoff=1.5)
plot_trace(signal_col,filtered_signal)

# Plot the frequency spectrum
fs=256
ecg_freq_content = ft.compute_dft()
freqs = np.fft.fftfreq(len(signal_col), d=1/fs)
plt.plot(freqs[:len(freqs)//2], np.abs(ecg_freq_content)[:len(freqs)//2])
plt.title("ECG Frequency Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.show()

## Hilbert Transform

In [ ]:
from vitalDSP.transforms.hilbert_transform import HilbertTransform
import numpy as np
import plotly.graph_objects as go
import os
from vitalDSP.notebooks import process_in_chunks, plot_trace


FILE_PATH = os.path.join("..","..","..","sample_data","public","ecg.csv")
signal_col, date_col = process_in_chunks(FILE_PATH,data_type='ecg', fs=256)
signal_col = np.array(signal_col)

# Apply Hilbert Transform
ht = HilbertTransform(signal_col)
analytic_signal = ht.compute_hilbert()
# Compute the envelope of the ECG signal
envelope_ecg = ht.envelope()
# Compute the instantaneous phase of the ECG signal
phase_ecg = ht.instantaneous_phase()
print("ECG Analytic Signal:", analytic_signal)
fig = go.Figure()
fig.add_trace(go.Scatter(y=envelope_ecg, mode="lines", name="envelope_ecg"))
fig.add_trace(go.Scatter(y=phase_ecg, mode="lines", name="phase_ecg"))
fig.show()

print("ECG Envelope:", envelope_ecg)
print("ECG Instantaneous Phase:", phase_ecg)
plot_trace(signal_col,np.real(analytic_signal))


# Vital Transform

In [ ]:
from vitalDSP.transforms.vital_transformation import VitalTransformation
import numpy as np
import matplotlib.pyplot as plt
import os
from vitalDSP.notebooks import process_in_chunks, plot_trace

fs = 256
signal_type = 'ecg'
FILE_PATH = os.path.join("..","..","..","sample_data","public","ecg.csv")
signal_col, date_col = process_in_chunks(FILE_PATH,data_type=signal_type, fs=fs)
signal_col = np.array(signal_col)

options = {
    'artifact_removal': 'baseline_correction',
    'artifact_removal_options': {'cutoff': 0.5},
    'bandpass_filter': {'lowcut': 0.2, 'highcut': 10, 'filter_order': 4, 'filter_type': 'butter'},
    'detrending': {'detrend_type': 'linear'},
    'normalization': {'normalization_range': (0, 20)},
    'smoothing': {'smoothing_method': 'moving_average', 'window_size': 5, 'iterations': 2},
    'enhancement': {'enhance_method': 'square'},
    'advanced_filtering': {'filter_type': 'kalman_filter', 'options': {'R': 0.1, 'Q': 0.01}},
    }
method_order = ['artifact_removal', 'bandpass_filter', 'detrending', 'normalization', 
                'smoothing', 'enhancement','advanced_filtering']
transformer = VitalTransformation(signal_col, fs=fs, signal_type=signal_type)
transformed_signal = transformer.apply_transformations(options, method_order)
plot_trace(signal_col, transformed_signal)